<a href="https://colab.research.google.com/github/spatank/CIS-522/blob/main/Homework/HW_6_SPP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CIS-522 Week 6 Homework

__Instructor__: Konrad Kording

__Content creators:__ Hmrishav Bandyopadhyay, Rahul Shekhar, Tejas Srivastava


In this homework, you will implement image classification problem on a much more complex Dataset, the **CIFAR 10 Dataset**. We will train two models a fully connected network and a Convolutional Neural Network and observe the differences in their performance on images. Further, we will also try to visualize the features learn by some of our Convolutional layers in CNN and answer some questions about ethics and our pod members!

In [ ]:
#@markdown What is your Pennkey and pod? (text, not numbers, e.g. bfranklin)
my_pennkey = '' #@param {type:"string"}
my_pod = 'Select' #@param ['Select', 'euclidean-wombat', 'sublime-newt', 'buoyant-unicorn', 'lackadaisical-manatee','indelible-stingray','superfluous-lyrebird','discreet-reindeer','quizzical-goldfish','ubiquitous-cheetah','nonchalant-crocodile','fashionable-lemur','spiffy-eagle','electric-emu','quotidian-lion','astute-jellyfish', 'quantum-herring']

# start timing
import time
try:t0;
except NameError: t0 = time.time()


In [ ]:
# @title Run this cell for imports.

# imports
import random
import pathlib

import torch
import numpy as np
import matplotlib.pyplot as plt

import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, TensorDataset
from torchvision.utils import make_grid
from IPython.display import HTML, display

from tqdm.notebook import tqdm, trange
from time import sleep

In [ ]:
# @markdown Figure Settings
%config InlineBackend.figure_format = 'retina'
%matplotlib inline 
plt.style.use("https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/nma.mplstyle")


In [ ]:
#@markdown Device Settings<br> It should print 'cuda', if not change your runtime to GPU.
device =  torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


# Part 0. Exploring the Dataset and Basic Functions





The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images. The classes in the dataset are:<br>
['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

Let us download the dataset, and build our datatloaders. Set the **BATCH_SIZE** for the dataloaders yourself. 

The three dataloaders are accessible by the variables *trainloader*, *validationloader* and *testloader*. 

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


# TODO: set Batch size for dataloaders
BATCH_SIZE = ...

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)


testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# splitting the testset into testset and validationset, with a 80:20 ratio
validationset, testset = torch.utils.data.random_split(testset, [8000, 2000])

validationloader = torch.utils.data.DataLoader(validationset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Let us check the sizes of our three sets and print some images. 

In [ ]:
print(len(trainset))
print(len(validationset))
print(len(testset))

In [ ]:
# function to show an image
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

dataiter = iter(trainloader)
images, labels = dataiter.next()
print(images.shape)

imshow(torchvision.utils.make_grid(images[0:4]))
print('    \t\t'.join('%5s' % classes[labels[j]] for j in range(4)))

Now let us write a train function which basically trains any network by running epochs and also evaluates the results on our validation set in every epoch. We will keep tunable hyperparameters as the parameters for the function, so that we can directly tweak them and call the function. 

In [ ]:
def train(model, device, train_loader, validation_loader, epochs, criterion, optimizer):
  # Params:
  #   model-> instance of your model which extends nn.Module class
  #   device -> it is set to cuda already and is already stored in a variable called *device*
  #   train_loader -> dataloader for train dataset, already stored in a variable called *trainloader*
  #   validation_loader -> dataloader for validation dataset, already stored in a variable called *validationloader*
  #   epochs -> number of epochs for training, you are expected to experiment and play around with this
  #   criterion -> the loss function which you decide to use
  #   optimizer -> optimizer for your gradient descent, do not forget to set an appropriate learning rate
  #  Returns:  train_loss, train_accuracy, validation_loss, validation_accuracy, all of these are lists storing the values 

    train_loss, validation_loss = [], []
    train_acc, validation_acc = [], []
    for epoch in range(epochs):
        model.train()
        running_loss = 0.
        correct, total = 0, 0 
        # TODO: train on the train dataset
        with tqdm(train_loader, unit='batch') as tepoch:
            tepoch.set_description('Training: ')
            ...
            
            

        
        
                
        # TODO: evaluate on validation data
        model.eval()
        running_loss = 0.
        correct, total = 0, 0 
        with tqdm(validation_loader, unit='batch') as tepoch:
            tepoch.set_description('Validation: ')
            ...
         
    
    return train_loss, train_acc, validation_loss, validation_acc 

Now, let us write a test function, which returns the accuracy of a trained model on the testdataset, given the testloader. 


In [ ]:
def test(model, device, data_loader):
  # Params:
  #   model-> instance of your trained model which extends nn.Module class
  #   device -> it is set to cuda already and is already stored in a variable called *device*
  #   data_loader -> dataloader for test dataset, already stored in a variable called *testloader*
  # Returns:
  #   accuracy(float)-> percentage of correct predictions made by your trained model
    model.eval()
    correct = 0
    total = 0
    for data in data_loader:
      # TODO: fill in the test function
      ...



    return acc

Below, we have provided a function to plot your train loss, test loss, train accuracy and test accuracy, during the training process. They take in arguments, which were the four lists returned by your train function. 

In [ ]:
# code to plot loss and accuracy
def plot_loss_accuracy(train_loss, train_acc, validation_loss, validation_acc):
    epochs = len(train_loss)
    fig, (ax1, ax2) = plt.subplots(1, 2)
    ax1.plot(list(range(epochs)), train_loss, label='Training Loss')
    ax1.plot(list(range(epochs)), validation_loss, label='Validation Loss')
    ax1.set_xlabel('Epochs')
    ax1.set_ylabel('Loss')
    ax1.set_title('Epoch vs Loss')
    ax1.legend()

    ax2.plot(list(range(epochs)), train_acc, label='Training Accuracy')
    ax2.plot(list(range(epochs)), validation_acc, label='Validation Accuracy')
    ax2.set_xlabel('Epochs')
    ax2.set_ylabel('Accuracy')
    ax2.set_title('Epoch vs Accuracy')
    ax2.legend()
    fig.set_size_inches(15.5, 5.5)
    plt.show()


# Part 1: FeedForward Neural Network

In this section, we will implement a Feed forward Neural Network that can classify the images in the CIFAR 10 dataset. 

Here, we define our FFN class which will consists of all the layers and the forward function. Feel free to play around by introducing different types of regularization, architectures, learning rates, epochs, and batch sizes, in order to achieve maximum accuracy, except the constraint that do not use convolutional layers in the model. 

## Architecture

In [ ]:
class FFN(nn.Module):
  def __init__(self):       
    super(FFN, self).__init__()
    # TODO: fill in the layers
    self.linear = ...

  # TODO: define the forward pass
  def forward(self, x):

    return x



## Training

Now, let us intantiate our FFN class, define the loss, the optimizer and call the train function. 

In [ ]:
# TODO: set criterion, optimizer and other hyperparameters and train
net1 = FFN().to(device)
criterion = ...
optimizer = ...
num_epochs = ...
train_loss, train_acc, validation_loss, validation_acc = train(net1, device, trainloader, validationloader, num_epochs, criterion, optimizer)

## Training Plots

Now, let us plot the train and validation curves. 

In [ ]:
plot_loss_accuracy(train_loss, train_acc, validation_loss, validation_acc)

Take a screenshot of your Training Curves (Both Accuracy and Loss) and submit it in the AirTable Form at the end of the Worksheet.

## Test Accuracy

And test our trained network on the test set. 

In [ ]:
test(net1, device, testloader)

And fill up the accuracy value you get. 

In [ ]:
ffn_accuracy = '''xx.xx %'''

Try to reach a test accuracy of atleast 45% with the Fully Connected Network. Try for even more!

You may try different models and variations and then submit the results (training curves) and the accuracy of your best model.

## Discuss



In [ ]:
#@markdown Now, discuss the hyperparameters and architectures you chose. Which model led to the best performance? Why? (200 words)
import time
try:t1;
except NameError: t1 = time.time()

In [ ]:
ffn_discussion = ''' 
write your answer here
'''

# Part 2: Convolutional Neural Network

Now, design a Convolutional Neural Network, for the same problem. Again, try to play around with various architectures, regularizations, activations, learning rates, optmizers and try to leverage the power of CNNs!

**Note:** Put all your Convolutional layers, activations in the first Sequential module, instead of defninig different layers and making an explicit forward pass.
Similarly, add all the subsequent fully connected layers in the second Sequential module, which comes after flattening the output from the convolutional layers.

**Do not change the forward pass code here.** 
<br>
This will help you run the next section on Visualization on your chosen architecture and visualize the feature maps of different convolutional layers. 

## Architecture

In [ ]:

class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            # TODO: fill the convolutional layers
            nn.Conv2d(...),
            nn.ReLU(),
            ...
        )

        self.fc = nn.Sequential(
            # TODO: fill the fully connected layers that follow the conv module
            nn.Linear(...),
            
         )
        
    # DO NOT CHANGE THE FORWARD PASS
    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        output=self.fc(x)
        return output

## Training

Calling the train function for the ConvNet.

In [ ]:
# TODO: fill the hyperparameters and train the model
net2 = CNN().to(device)
criterion = ...
optimizer = ...
num_epochs = ...
train_loss, train_acc, validation_loss, validation_acc = train(net2, device, trainloader, validationloader, num_epochs, criterion, optimizer)

## Training Plots

Now plot your training curves!

In [ ]:
plot_loss_accuracy(train_loss, train_acc, validation_loss, validation_acc)

## Test Accuracy

And, test your trained model on the Test Dataset. 

In [ ]:
test(net2, device, testloader)

And, fill up the accuracy here.

In [ ]:
cnn_accuracy = '''xx.xx %'''

Try to reach an accuracy of atleast 52%.Of course try to get your test accuracy even higher, and leverage the true powers of CNNs!

## Discussion

In [ ]:
#@markdown Discuss the hyperparameters and architectures you chose. Which model led to the best performance? Did your CNN outperform your FFN? Why? (200 words)
import time
try:t2;
except NameError: t2 = time.time()

In [ ]:
cnn_discussion = '''
Write your response here
'''

# Part 3: Visualizations: What do the features look like?

In [ ]:
dataiter = iter(trainloader)
images, labels = dataiter.next()
input_img=images[random.randrange(0,BATCH_SIZE,1)].unsqueeze(0)
img_original=input_img.squeeze().permute(1,2,0).numpy()


Layer_Name= 3 #@param {type:"slider", min:1, max:54, step:1}
Layer_Name=str(Layer_Name)
Num_Filters = 23 #@param {type:"slider", min:1, max:64, step:1}
from skimage.transform import resize
import matplotlib.pyplot as plt
def temp_rem(model,key,input_img):
    model.eval()
    flag=0
    alt_=[]    
    for search in model._modules['features']._modules.keys():
        if flag==1:
            break
        if key==search:
            flag=1
        alt_.append(model._modules['features']._modules[search])
    return torch.nn.Sequential(*alt_).to(device)(input_img.to(device)).squeeze().detach().cpu().numpy()

def plot(data,Num_Filters,img_original):
    import matplotlib.pyplot as plt
    i=0
    flag=0
    #print(len(data))
    if Num_Filters>len(data):
        print("Please set Num_Filters parameter less than or equal to the number of filters present in the layer ({} in this case)".format(len(data)))
        return
    if Num_Filters<8:
        fig, ax = plt.subplots(1,Num_Filters)
        for col in ax:
            if flag==0:
                col.imshow(resize(img_original,(256,256)))    
                col.set_xticklabels([])
                col.set_yticklabels([])
                
                flag=1
            else:
                col.imshow(resize(data[i],(256,256)))
                col.set_xticklabels([])
                col.set_yticklabels([])
                
            i=i+1
        
        plt.tight_layout()
        plt.subplots_adjust(wspace=0, hspace=0)
        plt.show()
        return

    i=0    
    if Num_Filters<16:
        ncols=4
    else:
        ncols=8

    nrows=Num_Filters//ncols
    fig, ax = plt.subplots(nrows=nrows, ncols=ncols)
    for rows in ax:
        for cols in rows:
            if flag==0:
                cols.imshow(resize(img_original,(256,256)))    
                cols.set_xticklabels([])
                cols.set_yticklabels([])
                flag=1
            else:
                cols.imshow(resize(data[i],(256,256)))
                cols.set_xticklabels([])
                cols.set_yticklabels([])
                
            i=i+1
        
        
    fig.set_size_inches(ncols*2.5,nrows*2.5)
    plt.tight_layout()
    plt.subplots_adjust(wspace=0, hspace=0)
    plt.show()
    

model = net2
plot(temp_rem(model,Layer_Name,input_img),Num_Filters,img_original)


## Discussion

 Describe some of the convolutional feature maps for certain images. What do you think they are representing? Do deeper convolutions result in different feature representations? (200 words)


In [ ]:
#@markdown Describe some of the convolutional feature maps for certain images. What do you think they are representing? Do deeper convolutions result in different feature representations? (200 words)
import time
try:t3;
except NameError: t3 = time.time()

In [ ]:
visualization_discuss = '''
Write your response here

'''

# Part 4: Deep Learning, Deep Thinking


## Part 4.1

Okay, so you all just solved a classification task using the CIFAR Dataset. Before we move ahead, please read the following to articles. 
 https://excavating.ai/ and https://medium.com/analytics-vidhya/an-introduction-to-fairness-in-machine-learning-62ef827e0020 (explains the fundamentals of measuring fairness in machine learning models).
 
 
<br><br> 
What do these readings suggest? There may be problems with unbalanced data and biases present in data. Now, go back and reflect on what you just did. Did you consider these problems when you designed your convnet? Write your thoughts (about 100 words).

In [ ]:
bias_response = '''
Your response here
'''

Now, its time to see what those implications are on the models you just trained. <br>

<br>



> Report the results of the independence and separation criteria you just read about in the medium article i.e report the accuracy for each class (independence) and report the false positive and false negative for each class (separation). <br>You can write separate code for implementing this. 
<br>
Submit a screenshot of your results in the AirTable Form at the end. (Do not worry about the format, it should be interpretable and readable)




In [ ]:
# Your code here



## Part 4.2

Let’s take a step back and consider the following questions 

1.   why might data be biased or imbalanced in the first place? Who gets to determine the labels of these data, and what might be some ethical concerns here?
2.   what are ethical implications of not investigating the data beforehand? Give a concrete example of some strategies to mitigate this.
<br><br>
Post your thoughts on your pods slacks channel and have a discussion amongst your pod members.

# Part 5: Better Know a Pod

Discuss with two other members of your pod. What is something they've always wanted to learn but have never gotten the chance to? (~100 words each)

In [ ]:
#@markdown Discuss with two other members of your pod. What is something they've always wanted to learn but have never gotten the chance to? (~100 words each)
import time
try:t4;
except NameError: t4 = time.time()

In [ ]:
know_a_pod_1 = '''
  First response here
'''

In [ ]:
know_a_pod_2 = '''
  Second Response here
'''

# Submit your responses
Please run the following cell and then press "Submit" so we can record your responses.

In [ ]:

#@markdown #Run Cell to Show Airtable Form
#@markdown ##**Confirm your answers, upload your screenshots, paste the notebook link and then click "Submit"**

import time
import numpy as np
import urllib.parse
from IPython.display import IFrame
def prefill_form(src, fields: dict):
  '''
  src: the original src url to embed the form
  fields: a dictionary of field:value pairs,
  e.g. {"pennkey": my_pennkey, "location": my_location}
  '''
  prefill_fields = {}
  for key in fields:
      new_key = 'prefill_' + key
      prefill_fields[new_key] = fields[key]
  prefills = urllib.parse.urlencode(prefill_fields)
  src = src + prefills
  return src


#autofill time if it is not present
try: t0;
except NameError: t0 = time.time()
try: t1;
except NameError: t1 = time.time()
try: t2;
except NameError: t2 = time.time()
try: t3;
except NameError: t3 = time.time()
try: t4;
except NameError: t4 = time.time()


#autofill fields if they are not present
#a missing pennkey and pod will result in an Airtable warning
#which is easily fixed user-side.
try: my_pennkey;
except NameError: my_pennkey = ""

try: my_pod;
except NameError: my_pod = "Select"

try: ffn_accuracy;
except NameError: ffn_accuracy = ""

try: ffn_discussion;
except NameError: ffn_discussion = ""

try: cnn_accuracy;
except NameError: cnn_accuracy = ""

try: cnn_discussion;
except NameError: cnn_discussion = ""

try: visualization_discuss;
except NameError: visualization_discuss = ""

try: know_a_pod_1;
except NameError: know_a_pod_1 = ""

try: know_a_pod_2;
except NameError: know_a_pod_2 = ""

try: bias_response;
except NameError: bias_response = ""

times = np.array([t1, t2,t3,t4])-t0

fields = {"my_pennkey": my_pennkey,
          "my_pod": my_pod,
          "ffn_accuracy":ffn_accuracy,
          "ffn_discussion": ffn_discussion,
          "cnn_accuracy": cnn_accuracy,
          "cnn_discussion": cnn_discussion,
          "visualization_discuss":visualization_discuss,
          "know_a_pod_1":know_a_pod_1,
          "know_a_pod_2":know_a_pod_2,
          "bias_response":bias_response,
          "cumulative_times": times}

src = "https://airtable.com/shrp0bcg2kvnLqKDN"
#now instead of the original source url, we do: src = prefill_form(src, fields)
display(IFrame(src = prefill_form(src, fields), width = 800, height = 400))